In [ ]:
!pip install opendatasets
!pip install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/gti-upm/leapgestrecog") # Download the dataset from Kaggle using opendatasets.


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: parthm
Your Kaggle Key: ··········


100%|██████████| 2.13G/2.13G [00:22<00:00, 102MB/s]


In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# Set the path where the downloaded dataset is stored. This is specific to the environment where you are running your notebook, such as Google Colab.
dataset_path = '/content/leapgestrecog/leapGestRecog/'

images = [] # Initialize an empty list to store image data.
labels = [] # Initialize an empty list to store labels corresponding to each image.

# Create a dictionary to map the folder names to a label. Each gesture type represented by a folder is mapped to a numerical label.
gesture_mapping = {'01_palm': 0, '02_l': 1, '03_fist': 2, '04_fist_moved': 3,
                   '05_thumb': 4, '06_index': 5, '07_ok': 6, '08_palm_moved': 7,
                   '09_c': 8, '10_down': 9}

In [ ]:
# Iterate over each folder in the dataset directory. Each folder corresponds to a different subject.
for subject_folder in os.listdir(dataset_path):
    subject_path = os.path.join(dataset_path, subject_folder) # Combine the base path with the subject folder path.
    if not os.path.isdir(subject_path):
    if not os.path.isdir(subject_path):
        continue # Skip the iteration if the path does not lead to a directory.
    # Iterate over each gesture type specified in the gesture_mapping dictionary.
    for gesture in gesture_mapping.keys():
        path = os.path.join(subject_path, gesture) # Construct the path to the gesture images.
        # Iterate over each image file within the gesture folder.
        for img_file in os.listdir(path):
            img_path = os.path.join(path, img_file)

            # Read the image in grayscale and resize it to 128x128 pixels, then normalize the pixel values.
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (128, 128)) / 255.0
            # Append the processed image to the images list and its corresponding label to the labels list.
            images.append(image)
            labels.append(gesture_mapping[gesture])

# Convert the images and labels lists to numpy arrays. Reshape images to add a channel dimension, required for CNN input.
images = np.array(images).reshape(-1, 128, 128, 1)
labels = to_categorical(labels) # Convert labels to one-hot encoded format.

In [ ]:
# Split the dataset into training and testing sets with 20% of data reserved as the test set.
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define a Sequential model with CNN layers for feature extraction and Dense layers for classification.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [ ]:
# Compile the model with the Adam optimizer and categorical crossentropy loss function, tracking accuracy.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data, validating on the testing data, for 10 epochs.
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Save the trained model to a file for later use or deployment.
model.save('gesture_model.keras')